In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

## The data, split between train and test sets:

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Convert class vectors to binary class matrices

In [5]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Making Layers For Network

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

## initiate RMSprop optimizer

In [8]:
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

## Let's train the model using RMSprop optimizer

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [10]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [11]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

Using real-time data augmentation.


### Compute quantities required for feature-wise normalization
    (std, mean, and principal components if ZCA whitening is applied)

In [12]:
datagen.fit(x_train)

### Fit the model on the batches generated by datagen.flow()

In [13]:
model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Epoch 1/100
1563/1563 [==============================] - 125s 80ms/step - loss: 1.8658 - accuracy: 0.3100 - val_loss: 1.5109 - val_accuracy: 0.4522
Epoch 2/100
1563/1563 [==============================] - 122s 78ms/step - loss: 1.5703 - accuracy: 0.4276 - val_loss: 1.3676 - val_accuracy: 0.5049
Epoch 3/100
1563/1563 [==============================] - 117s 75ms/step - loss: 1.4644 - accuracy: 0.4715 - val_loss: 1.2979 - val_accuracy: 0.5324
Epoch 4/100
1563/1563 [==============================] - 116s 74ms/step - loss: 1.3910 - accuracy: 0.5024 - val_loss: 1.2470 - val_accuracy: 0.5550
Epoch 5/100
1563/1563 [==============================] - 118s 75ms/step - loss: 1.3308 - accuracy: 0.5242 - val_loss: 1.1880 - val_accuracy: 0.5773
Epoch 6/100
1563/1563 [==============================] - 131s 84ms/step - loss: 1.2735 - accuracy: 0.5464 - val_loss: 1.2003 - val_accuracy: 0.5736
Epoch 7/100
1563/1563 [==============================] - 130s 83ms/step - loss: 1.2262 - accuracy: 0.5644 - val_

1563/1563 [==============================] - 119s 76ms/step - loss: 0.7871 - accuracy: 0.7346 - val_loss: 0.7002 - val_accuracy: 0.7708
Epoch 57/100
1563/1563 [==============================] - 119s 76ms/step - loss: 0.7892 - accuracy: 0.7352 - val_loss: 0.6919 - val_accuracy: 0.7704
Epoch 58/100
1563/1563 [==============================] - 118s 76ms/step - loss: 0.7829 - accuracy: 0.7359 - val_loss: 0.7398 - val_accuracy: 0.7569
Epoch 59/100
1563/1563 [==============================] - 118s 76ms/step - loss: 0.7876 - accuracy: 0.7344 - val_loss: 0.6800 - val_accuracy: 0.7865
Epoch 60/100
1563/1563 [==============================] - 119s 76ms/step - loss: 0.7833 - accuracy: 0.7375 - val_loss: 0.6725 - val_accuracy: 0.7774
Epoch 61/100
1563/1563 [==============================] - 118s 76ms/step - loss: 0.7877 - accuracy: 0.7378 - val_loss: 0.7295 - val_accuracy: 0.7685
Epoch 62/100
1563/1563 [==============================] - 119s 76ms/step - loss: 0.7784 - accuracy: 0.7394 - val_loss: 

### Saving model and weights

In [ ]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

### Evaluating model on test data set

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])